In [1]:
pip install transformers torch speechrecognition gtts PyMuPDF sentencepiece pydub


  Using cached speechrecognition-3.14.3-py3-none-any.whl.metadata (30 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached speechrecognition-3.14.3-py3-none-any.whl (32.9 MB)
Using cached sentencepiece-0.2.0-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [speechrecognition]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # open the PDF file
    text = ""
    for page in doc:  # iterate through each page
        text += page.get_text()  # extract text
    doc.close()
    return text


In [4]:
import whisper
import pyaudio
import wave

def record_audio(filename="question.wav", record_seconds=5, rate=16000):
    chunk = 1024
    fmt = pyaudio.paInt16
    channels = 1

    p = pyaudio.PyAudio()
    stream = p.open(format=fmt, channels=channels,
                    rate=rate, input=True,
                    frames_per_buffer=chunk)
    print("Recording your voice (speak now)...")

    frames = []
    for _ in range(0, int(rate / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)

    print("Recording complete.")
    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(fmt))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

def get_voice_input_whisper():
    record_audio()
    model = whisper.load_model("base")
    result = model.transcribe("question.wav")
    print("You asked:", result["text"])
    return result["text"]


In [5]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

def answer_with_bert(context, question):
    tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    )
    return answer


/Users/abhijith/Christ/LLM/venv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from gtts import gTTS
import os

def speak_text(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("answer.mp3")
    os.system("start answer.mp3")  # 'afplay' for macOS, 'xdg-open' for Linux


In [7]:
# Step 1: Load PDF in English
context_en = extract_text_from_pdf("english_doc.pdf")

# Step 2: Ask question via voice (Whisper)
question_en = get_voice_input_whisper()

# Step 3: QA using BERT (no pipeline)
answer_en = answer_with_bert(context_en, question_en)
print("Answer:", answer_en)

# Step 4: Speak the answer
speak_text(answer_en, lang='en')


Recording your voice (speak now)...
Recording complete.


/Users/abhijith/Christ/LLM/venv310/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


You asked:  Christ, the interviewee universe.


Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [21]:
import whisper
print(dir(whisper))


['AUTOFLUSH', 'BUFFERING', 'CACHE_HEADERS', 'CAN_FADVISE', 'CAN_FALLOCATE', 'CAN_LOCK', 'CorruptWhisperFile', 'FADVISE_RANDOM', 'InvalidAggregationMethod', 'InvalidConfiguration', 'InvalidTimeInterval', 'InvalidXFilesFactor', 'LOCK', 'TimestampNotCovered', 'UnitMultipliers', 'WhisperException', '__archive_fetch', '__archive_update_many', '__builtins__', '__cached__', '__doc__', '__file__', '__headerCache', '__loader__', '__name__', '__package__', '__propagate', '__readHeader', '__setAggregation', '__spec__', '__writeHeaderMetadata', 'aggregate', 'aggregationMethodToType', 'aggregationMethods', 'aggregationTypeToMethod', 'archiveInfoFormat', 'archiveInfoSize', 'c_off64_t', 'c_off_t', 'create', 'ctypes', 'debug', 'diff', 'disableDebug', 'enableDebug', 'endBlock', 'fallocate', 'fcntl', 'fetch', 'file_diff', 'file_fetch', 'file_merge', 'file_update', 'file_update_many', 'floatFormat', 'floatSize', 'getUnitString', 'ifilter', 'info', 'itertools', 'izip', 'longFormat', 'longSize', 'merge', '

In [ ]:
import os
import platform
import subprocess
from gtts import gTTS

# Function 1: Extract PDF
def extract_text_from_pdf(pdf_path):
    import fitz
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function 2: Record + Transcribe with Whisper
def get_voice_input_whisper():
    import whisper
    import pyaudio
    import wave

    def record_audio(filename="question.wav", record_seconds=5, rate=16000):
        chunk = 1024
        fmt = pyaudio.paInt16
        channels = 1
        p = pyaudio.PyAudio()
        stream = p.open(format=fmt, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)
        print("Recording your voice (speak now)...")

        frames = []
        for _ in range(0, int(rate / chunk * record_seconds)):
            data = stream.read(chunk)
            frames.append(data)

        print("Recording complete.")
        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(filename, 'wb')
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(fmt))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))
        wf.close()

    record_audio()
    model = whisper.load_model("base")
    result = model.transcribe("question.wav")
    print("You asked:", result["text"])
    return result["text"]

# Function 3: QA with BERT
def answer_with_bert(context, question):
    from transformers import BertTokenizer, BertForQuestionAnswering
    import torch
    tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    start = torch.argmax(outputs.start_logits)
    end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start:end])
    )
    return answer

# Function 4: Speak the Answer (clean exit)
def speak_text(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("answer.mp3")

    system = platform.system()
    try:
        if system == "Darwin":  # macOS
            subprocess.run(["afplay", "answer.mp3"])
        elif system == "Linux":
            subprocess.run(["xdg-open", "answer.mp3"])
        elif system == "Windows":
            subprocess.run(["start", "answer.mp3"], shell=True)
        else:
            print("Unsupported OS")
    except Exception as e:
        print("Error playing audio:", e)
    finally:
        if os.path.exists("answer.mp3"):
            os.remove("answer.mp3")  # optional cleanup


if __name__ == "__main__":
    context_en = extract_text_from_pdf("english_doc.pdf")
    question_en = get_voice_input_whisper()
    answer_en = answer_with_bert(context_en, question_en)
    print("Answer:", answer_en)
    speak_text(answer_en, lang='en')

    
    import sys
    sys.exit(0)



Recording your voice (speak now)...
Recording complete.


/Users/abhijith/Christ/LLM/venv310/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


You asked:  Hello, how are you? You said, damn it.


In [15]:
import torch
import sounddevice as sd
import numpy as np
import whisper
import PyPDF2
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from gtts import gTTS
import os
import wave

In [16]:
# ----------------------------
# Utility: PDF Extraction
# ----------------------------
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

In [17]:
# ----------------------------
# Utility: Record voice (as .wav)
# ----------------------------
def record_audio(filename="question.wav", duration=5, fs=44100):
    print("🎙️ Recording your voice (speak now)...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    with wave.open(filename, 'w') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(recording.tobytes())
    print("✅ Recording saved as", filename)

In [18]:
# ----------------------------
# Utility: Whisper for STT
# ----------------------------
def get_voice_input_whisper(filename="question.wav"):
    record_audio(filename)
    model = whisper.load_model("base")
    result = model.transcribe(filename)
    print("🗣️ You asked:", result["text"])
    return result["text"]

In [19]:
# ----------------------------
# QA Model Loader (not pipeline)
# ----------------------------
def load_qa_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    return tokenizer, model

In [20]:
# ----------------------------
# QA Inference (no pipeline)
# ----------------------------
def answer_question(tokenizer, model, context, question):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    start = torch.argmax(outputs.start_logits)
    end = torch.argmax(outputs.end_logits) + 1
    return tokenizer.decode(inputs['input_ids'][0][start:end], skip_special_tokens=True)

In [21]:
# ----------------------------
# Speak output
# ----------------------------
def speak_text(text, lang='en'):
    tts = gTTS(text, lang=lang)
    tts.save("response.mp3")
    os.system("start response.mp3" if os.name == "nt" else "mpg123 response.mp3")

In [24]:
# ----------------------------
# Complete QA Flow for any language
# ----------------------------
def qa_flow(language):
    if language == 'english':
        print("\n=== English ===")
        context = extract_text_from_pdf("english_doc.pdf")
        tokenizer, model = load_qa_model("bert-base-uncased")
        tts_lang = "en"
    elif language == 'hindi':
        print("\n=== Hindi ===")
        context = extract_text_from_pdf("hindi_doc.pdf")
        tokenizer, model = load_qa_model("ai4bharat/indic-bert")
        tts_lang = "hi"
    elif language == 'french':
        print("\n=== French ===")
        context = extract_text_from_pdf("french_doc.pdf")
        tokenizer, model = load_qa_model("camembert-base")
        tts_lang = "fr"
    else:
        print("Unsupported language.")
        return

    question = get_voice_input_whisper()
    answer = answer_question(tokenizer, model, context, question)
    print("📘 Answer:", answer)
    speak_text(answer, lang=tts_lang)

In [26]:
# ----------------------------
# Main program
# ----------------------------
if __name__ == "__main__":
    for lang in ["english", "hindi", "french"]:
        qa_flow(lang)


=== English ===


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🎙️ Recording your voice (speak now)...
||PaMacCore (AUHAL)|| Warning on line 521: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Warning on line 441: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Error on line 1322: err='-10851', msg=Audio Unit: Invalid Property Value


PortAudioError: Error opening InputStream: Internal PortAudio error [PaErrorCode -9986]

In [27]:
import sounddevice as sd
print(sd.query_devices())


> 0 External Microphone, Core Audio (1 in, 0 out)
< 1 External Headphones, Core Audio (0 in, 2 out)
  2 MacBook Air Microphone, Core Audio (1 in, 0 out)
  3 MacBook Air Speakers, Core Audio (0 in, 2 out)
